In [16]:
%load_ext micropython_magic
%reload_ext micropython_magic

The micropython_magic extension is already loaded. To reload it, use:
  %reload_ext micropython_magic


In [17]:
%mpy -s {"/dev/cu.usbmodem_fs3_1"}

In [18]:
# %%micropython

import math
import time
from config import config
from machine import RTC
from mp_libs import logging
from mp_libs.mpy_decimal import DecimalNumber
from mp_libs.network import Network
from mp_libs.protocols.min_iot_protocol import MinIotMessage
from mp_libs.time import ptp
from mp_libs.time.time import get_fmt_datetime

TIMEOUT_MSEC = 5000

rtc = RTC()
mqtt_network = Network.create_mqtt("GTY")
miniot_network = Network.create_min_iot("GTY")
epn = miniot_network.transport.transport.transport

# Connect networks and perform initial time sync
miniot_network.connect()
mqtt_network.connect()
mqtt_network.ntp_time_sync()

print(f"Initial RTC datetime: {rtc.datetime()}")
print(f"Initial RTC now: {rtc.now()}")
print(f"My MAC: {epn.wifi._sta.config('mac')}")
print(f"EPN Peer: {config['epn_peer_mac']}")

while True:
    data_available = False
    rxed_msgs: list[MinIotMessage] = []

    # Receive data
    while not data_available:
        data_available = miniot_network.receive(rxed_msgs)
        time.sleep_ms(1)

    for msg in rxed_msgs:
        # Handle PTP messages
        if ptp.is_ptp_msg(msg.msg):
            ptp_type, payload = ptp.parse_msg(msg.msg)

            if ptp_type != ptp.PtpMsg.SYNC_REQ:
                print("Rx'ed unexpected PTP message")
                continue

            ptp.sequence_master(
                miniot_network,
                lambda miniot_msg: miniot_msg.msg,
                TIMEOUT_MSEC,
                num_sync_cycles=payload)

            print(f"now: {rtc.datetime()}")
        else:
            print(msg)

2000-01-01 00:00:04.4503 INFO-wifi-protocols:Disconnecting wifi...
2000-01-01 00:00:04.4511 INFO-wifi-protocols:Disconnecting wifi...
2000-01-01 00:00:04.4513 DEBUG-espnow-protocol:espnow configured
2000-01-01 00:00:04.4516 INFO-miniot:Connecting MinIoT...
2000-01-01 00:00:04.4518 INFO-espnow-protocol:Connecting espnow...
2000-01-01 00:00:04.4520 INFO-wifi-protocols:Connecting wifi...
2000-01-01 00:00:04.4522 DEBUG-wifi-protocols:Connecting to AP: D&A_Home
2000-01-01 00:00:05.5916 INFO-wifi-protocols:Wifi is connected: ('192.168.0.109', '255.255.255.0', '192.168.0.1', '192.168.0.1')
2000-01-01 00:00:05.5917 INFO-wifi-protocols:Connecting mqtt...
2000-01-01 00:00:05.5919 DEBUG-mqtt:Attempting to connect to MQTT broker (attempt #0)
2000-01-01 00:00:05.5921 DEBUG-mqtt:Attempting to establish MQTT connection...
2000-01-01 00:00:05.5923 INFO-mqtt:Establishing an INSECURE connection to 192.168.0.19:1883
2000-01-01 00:00:05.5942 DEBUG-mqtt:Sending CONNECT to broker...
2000-01-01 00:00:05.5944